In [1]:
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.9 MB 1.2 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [1]:
import requests
import pandas as pd
import time
import re
import datetime as dt
import spacy
import praw
from psaw import PushshiftAPI

In [2]:
spacy_eng = spacy.load('en_core_web_sm')

In [3]:
max_len = 250
min_submission_score = 3

# min_comment_score = 2

# min_submission_score = 1
min_comment_score = 1

In [4]:
def text_cleaner (text):
    clean_text = text.split("update")[0]
    clean_text = clean_text.split("Update")[0]
    clean_text = clean_text.split("summary")[0]
    clean_text = clean_text.split("Summary")[0]
    clean_text = clean_text.split("edit")[0]
    clean_text = clean_text.split("Edit")[0]
    clean_text = clean_text.split("tl;dr")[0]
    clean_text = clean_text.split("Tl;dr")[0]
    clean_text = clean_text.replace("&#x200B;", " ")
    clean_text = clean_text.replace("&amp;#x200B;", " ")
    clean_text = clean_text.replace("\n", " ")
    clean_text = clean_text.replace("&gt;", ">")
    clean_text = clean_text.replace("'", "")
    clean_text = clean_text.replace("’", "")
    clean_text = clean_text.strip()
    re.sub(r'http\S+', ' ', clean_text)


    return clean_text

In [32]:
total_submissions = []

In [39]:
day_delta = dt.timedelta(days=1)

start_date = dt.datetime(2021, 5, 23)
end_date = dt.datetime(2021, 12, 31)

subreddit = 'SuicideWatch'

In [40]:
for i in range((end_date - start_date).days):      
    from_query_date = int((start_date + i*day_delta).timestamp())
    to_query_date = int((start_date + (i+1)*day_delta).timestamp())

    query = f"https://api.pushshift.io/reddit/submission/search/?after={from_query_date}&before={to_query_date}&subreddit={subreddit}&size=400"

    response = requests.get(query)

    if response.status_code == 200:
        submissions = response.json()['data']

        for submission in submissions:
            if 'selftext' in submission.keys() and len(submission['selftext']) > 0 and submission['selftext'] != '[deleted]' and submission['selftext'] != '[removed]' and submission['score'] > min_submission_score - 1:
                if submission['title'][-1] == '?' or submission['title'][-1] == '.':
                    submission_text = submission['title'] + " " + submission['selftext']
                else:
                    submission_text = submission['title'] + ". " + submission['selftext']


                submission_text = text_cleaner(submission_text)
                                
                if len([tok.text.lower() for tok in spacy_eng.tokenizer(submission_text)]) <= max_len:
                    total_submissions.append([submission['id'], submission_text, submission['score']])

    else:
        print(f"Response status code error {response.status_code} on {start_date + i*day_delta}")

    if i % 25 == 0:
        print(f"Sleeping... Day {i}")
        print(f"Total submissions by {start_date + i*day_delta}: {len(total_submissions)}")

        time.sleep(60)


Sleeping... Day 0
Total submissions by 2021-05-23 00:00:00: 50966
Sleeping... Day 25
Total submissions by 2021-06-17 00:00:00: 51076
Sleeping... Day 50
Total submissions by 2021-07-12 00:00:00: 51162
Sleeping... Day 75
Total submissions by 2021-08-06 00:00:00: 51162
Sleeping... Day 100
Total submissions by 2021-08-31 00:00:00: 51198
Sleeping... Day 125
Total submissions by 2021-09-25 00:00:00: 51219
Sleeping... Day 150
Total submissions by 2021-10-20 00:00:00: 51265
Sleeping... Day 175
Total submissions by 2021-11-14 00:00:00: 51287
Sleeping... Day 200
Total submissions by 2021-12-09 00:00:00: 51330


In [42]:
reddit = praw.Reddit(client_id='sxNv0-5xhcGU-VuK7cSPNw',
                     client_secret='vnJrB-DFLHQrksFX2akRvKtWDRTvhg',
                     user_agent='MentalHealthApp')

In [43]:
api = PushshiftAPI(reddit)

In [40]:
# total_submissions = []

In [41]:
# day_delta = dt.timedelta(days=1)

# start_date = dt.datetime(2021, 1, 1)
# end_date = dt.datetime(2021, 12, 30)

In [57]:
# 1020 330

In [58]:
# for i in range((end_date - start_date).days):      
#     from_query_date = int((start_date + i*day_delta).timestamp())
#     to_query_date = int((start_date + (i+1)*day_delta).timestamp())


#     submissions = list(api.search_submissions(after=from_query_date, before=to_query_date, subreddit='depression', filter=['url','author', 'title']))

#     submissions_generator = (submission for submission in submissions if len(submission.selftext) > 0 if len(submission.selftext) < max_len if submission.score > 2 if submission.selftext != '[deleted]' if submission.selftext != '[removed]')
#     submissions = list(submissions_generator)
    
#     if len(submissions) > 0:
#         total_submissions.extend(submissions)
    
#     if i % 30 == 0:
#         print(f"Sleeping...Day {i}")
#         print(f"Total submissions by {start_date + i*day_delta}: {len(total_submissions)}")
#         time.sleep(60)
    

Sleeping...Day 0
Total submissions by 2021-01-01 00:00:00: 64565
Sleeping...Day 30
Total submissions by 2021-01-31 00:00:00: 65814
Sleeping...Day 60
Total submissions by 2021-03-02 00:00:00: 66820
Sleeping...Day 90
Total submissions by 2021-04-01 00:00:00: 67490
Sleeping...Day 120
Total submissions by 2021-05-01 00:00:00: 68550
Sleeping...Day 150
Total submissions by 2021-05-31 00:00:00: 69904
Sleeping...Day 180
Total submissions by 2021-06-30 00:00:00: 71104
Sleeping...Day 210
Total submissions by 2021-07-30 00:00:00: 72425
Sleeping...Day 240
Total submissions by 2021-08-29 00:00:00: 73643


ServerError: received 503 HTTP response

In [44]:
len(total_submissions)

51330

In [45]:
import joblib

joblib.dump(total_submissions, 'total_submissions_suicidewatch.pkl') 


# total_submissions = joblib.load('total_submissions_suicidewatch.pkl') 

['total_submissions_suicidewatch.pkl']

In [46]:
total_submissions[1]

['8ugho',
 'This is very difficult for me. The one person whom I love cuts herself. This is an alt account. She reads reddit, as do I. I dont know what she would think if I told the whole world her story. But Im sure she would do the same as I am doing now if our places were switched.   The girl whom I love suffers from bipolar disorder. She often cuts herself and contemplates suicide whenever shes depressed. Shes received help from medical professionals and takes medication for her illness, but there are simply times when the illness overcomes her and she takes the knife to her arms.   Besides being supportive of her all the time, is there anything I can do  for her?   She once told me shed kill herself if anything happened to me. Those words touched me in so many ways. Yet, she sometimes says that she doesnt feel like living. I want to help her because I care about her.   Talk to me some more and ask any questions, and Ill provide you with more details if need be. I just need help wi

In [47]:
submission_comment_pairs = []

In [48]:
# https://api.pushshift.io/reddit/comment/search/?link_id=bc99el&limit=20

In [51]:
req_count = 0

for submission in total_submissions[44625+2450:]:
    submission_id = submission[0]
    
    query = f"https://api.pushshift.io/reddit/comment/search/?subreddit={subreddit}&link_id={submission_id}"

    try:
        response = requests.get(query)
        
        req_count += 1
        if response.status_code == 200:
            comments = response.json()['data']

            comments_generator = (comment for comment in comments if comment['body'] != '[deleted]' if comment['body'] != '[removed]' if comment['parent_id'].split("_")[-1] == submission_id if len([tok.text.lower() for tok in spacy_eng.tokenizer(text_cleaner(comment['body']))]) <= max_len)
            comments = list(comments_generator)
            
            if len(comments) > 0:
                comment = max(comments, key=lambda x:x['score'])

                comment_body = text_cleaner(comment['body'])

                submission_comment_pairs.append([submission[1], comment_body, submission[2], comment['score']])
            
        else:
            print(response)
   
    except requests.exceptions.RequestException as e:  # This is the correct syntax
#         raise SystemExit(e)
            print(e)

    
    if req_count % 25 == 0:
        print(f"Sleeping...ID: { str(req_count)}/{str(len(total_submissions))} Current Pair # " + str(len(submission_comment_pairs)))
        time.sleep(60)

Sleeping...ID: 25/51330 Current Pair # 39488
Sleeping...ID: 50/51330 Current Pair # 39512
Sleeping...ID: 75/51330 Current Pair # 39534
Sleeping...ID: 100/51330 Current Pair # 39557
Sleeping...ID: 125/51330 Current Pair # 39581
Sleeping...ID: 150/51330 Current Pair # 39604
Sleeping...ID: 175/51330 Current Pair # 39628
Sleeping...ID: 200/51330 Current Pair # 39653
Sleeping...ID: 225/51330 Current Pair # 39677
Sleeping...ID: 250/51330 Current Pair # 39701
<Response [502]>
Sleeping...ID: 275/51330 Current Pair # 39724
Sleeping...ID: 300/51330 Current Pair # 39747
Sleeping...ID: 325/51330 Current Pair # 39770
Sleeping...ID: 350/51330 Current Pair # 39793
Sleeping...ID: 375/51330 Current Pair # 39816
Sleeping...ID: 400/51330 Current Pair # 39840
Sleeping...ID: 425/51330 Current Pair # 39865
Sleeping...ID: 450/51330 Current Pair # 39889
Sleeping...ID: 475/51330 Current Pair # 39911
Sleeping...ID: 500/51330 Current Pair # 39936
Sleeping...ID: 525/51330 Current Pair # 39961
Sleeping...ID: 550/5

In [ ]:
req_count = 0
for submission in total_submissions:
    submission_id = submission[0]

    comments = list(api.search_comments(subreddit='depression', submission_id = submission_id, sort='score', limit=10))
    print(comments)
    comments_generator = (comment for comment in comments if comment.body != '[deleted]' if comment.body != '[removed]' if comment.parent_id.split("_")[-1] == submission_id)
    comments = list(comments_generator)

    req_count +=1

    if len(comments) > 0:
        comment = max(comments, key=lambda x:x.score)

        comment_body = text_cleaner(comment.body)
        
        print(len([tok.text.lower() for tok in spacy_eng.tokenizer(comment_body)]))
        
        if len([tok.text.lower() for tok in spacy_eng.tokenizer(comment_body)]) <= max_len:

            submission_comment_pairs.append([submission[1], comment_body, submission[2], comment.score])


    if req_count % 25 == 0:
        print(f"Sleeping...ID: { str(req_count)}/{str(len(total_submissions))} Current Pair # " + str(len(submission_comment_pairs)))
        time.sleep(60)

In [52]:
len(submission_comment_pairs)

43323

In [53]:
df = pd.DataFrame(submission_comment_pairs, columns = ['Submission', 'Comment', 'SubmissionScore', 'CommentScore'])

In [54]:
df["SubmissionScore"][0]

8

In [56]:
df.to_csv("submission_comment_score_pairs_suicide_watch.csv", index=False)